## Association Rule Learning Homework

*Prepared by:*
**Eduardo F. Valdez**  
Faculty, Software Technology Department  
College of Computer Studies - De La Salle University

---
You may do this solo or submit as a group using the MCO/project groupings.


Submitted by:
- **Eula Kathrina M. Tizon**

## Requirements

Answer the following questions:

**Groceries Dataset**  
1. Calculate the association rules and find the `significant/interesting` items in this dataset. Feel free to use any threshold value. (10 pts)
    - group the items by member
    - group the items by month
2. What would you recommend to the owner of a grocery store given these association rules? (15 pts)
3. Is there any other grouping that could give us high confidence/interest? (10 pts)

**Another Dataset**  
1. Find another dataset that can be turned into an association rule learning problem. Ideally, more than 1000 transactions/records. This does not have to be grocery-related. Create a data dictionary. Show basic statistics (e.g mean, median, mode, non-null and null values etc.)(20 pts)
2. Similar to the grocery dataset, calculate the association rules and find the `significant/interesting` items. (10 pts)
3. What would you recommend given the association rules? (25 pts)
4. Any other learning?

Note:
 - You have to create a writeup on the code you've written. Submit the writeup as a PDF file. Make sure that the code is clean as well. Moroever, there is documentation/text cells in the notebook. (5 pts)
 - Cite references if there are any. If you will be using the code I've given, cite it. If you got help from your peers, cite it as well. (5 pts)
 - Submit the link for your copy of the Google Colaboratory notebook. **Make sure it's public**. You may use incognito to see if the link is accessible.

<hr>

## Grading Criteria

|Criteria|Points|
|---|---|
|Groceries Dataset Analysis|35|
|Another Dataset Analysis|55|
|Code Cleanliness|5|
|References|5|


## Preliminaries

### Import Libraries

We will be using the `PyFIM` library to do association rule learning in this homework. If this is not already installed in your environment, you may use the either of the following commands in your command line:

```conda install -c conda-forge pyfim``` or
```pip install pyfim```

In [ ]:
!pip install pyfim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyfim: filename=pyfim-6.28-cp310-cp310-linux_x86_64.whl size=644243 sha256=02bb7da2543b6e8912cb102e274f6a8e34ec117ccca7155493d3266af8bba819
  Stored in directory: /root/.cache/pip/wheels/96/0a/b3/c877bfa85c4cfe1baf3de4a89e1949382be09de5eabe49314f
Successfully built pyfim


In [ ]:
import pandas as pd
from fim import arules, apriori, fpgrowth

## Groceries Dataset

### Prepare the Dataset

We will be utilizing the <a href="https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset">Groceries dataset</a> from Kaggle.

In [ ]:
df1 = pd.read_csv('https://raw.githubusercontent.com/efvaldez1/data-repository/main/groceries.csv')
df1.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


By working with this dataset, we can uncover purchasing patterns and associations between different items bought by customers.

### Data Preprocessing

#### Convert Date Column to DateTime
We convert the 'Date' column from string format to DateTime objects using the pd.to_datetime function. This conversion is essential for accurate date and time manipulation, allowing us to perform time-based analyses and groupings.

In [ ]:
df1['Date'] = pd.to_datetime(df1['Date'])

<ipython-input-4-695a7991ba53>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['Date'] = pd.to_datetime(df1['Date'])


#### Extract Month and Year
We extract the month from the 'Date' column and create a new column 'Month_Year'. This new column helps us group transactions by month, which is crucial for identifying seasonal trends and changes in purchasing behavior over time.

In [ ]:
df1['Month_Year'] = df1['Date'].dt.to_period('M')
df1.head()

,Member_number,Date,itemDescription,Month_Year
0,1808,2015-07-21,tropical fruit,2015-07
1,2552,2015-01-05,whole milk,2015-01
2,2300,2015-09-19,pip fruit,2015-09
3,1187,2015-12-12,other vegetables,2015-12
4,3037,2015-02-01,whole milk,2015-02


### Grouping Items

#### Group Transactions by Member and Month
We will group the items bought by each member for each month. This grouping allows us to aggregate the transactions and analyze the purchasing patterns of individual members over time. By using the groupby method, we'll generate a list of items for each combination of 'Member_number' and 'Month_Year', providing a comprehensive view of monthly purchasing habits.

In [ ]:
grouped_member_month1 = df1.groupby(['Member_number', 'Month_Year'])['itemDescription'].apply(list).reset_index()
grouped_member_month1

,Member_number,Month_Year,itemDescription
0,1000,2014-06,"[whole milk, pastry, salty snack]"
1,1000,2015-03,"[sausage, whole milk, semi-finished bread, yog..."
2,1000,2015-05,"[soda, pickled vegetables]"
3,1000,2015-07,"[canned beer, misc. beverages]"
4,1000,2015-11,"[sausage, hygiene articles]"
...,...,...,...
13900,4999,2015-05,"[butter milk, whipped/sour cream]"
13901,4999,2015-12,"[bottled water, herbs]"
13902,5000,2014-03,"[fruit/vegetable juice, onions]"
13903,5000,2014-11,"[bottled beer, other vegetables]"


The grouping resulted in 13905 rows.

### Calculate the Association rules

#### Generate Frequent Itemsets
We use the apriori algorithm from pyfim to generate frequent itemsets from the grouped transactions. We set the minimum support threshold to 3 to ensure that only itemsets appearing in at least 3% of all the transactions are considered. This threshold helps us focus on meaningful associations rather than insignificant itemsets.

In [ ]:
itemsets1 = apriori(grouped_member_month1['itemDescription'], supp = 3)
df1_itemsets = pd.DataFrame(itemsets1, columns = ['itemset', 'absolute support'])
df1_itemsets = df1_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df1_itemsets.shape)

df1_itemsets

(28, 2)


,itemset,absolute support
0,"(whole milk,)",2335
1,"(other vegetables,)",1805
2,"(rolls/buns,)",1628
3,"(soda,)",1442
4,"(yogurt,)",1276
5,"(root vegetables,)",1037
6,"(tropical fruit,)",1009
7,"(bottled water,)",903
8,"(sausage,)",892
9,"(citrus fruit,)",792


#### Generate Association Rules
Next, we'll generate association rules from the frequent itemsets using the arules function from pyfim. We'll set the minimum support to 3 and the minimum confidence to 10 to focus on meaningful associations. The resulting rules will be stored in a DataFrame with columns for consequent, antecedent, support, and confidence values. These rules help us understand the likelihood of items being purchased together and provide insights into customer purchasing behavior.

In [ ]:
result1 = arules(grouped_member_month1['itemDescription'], supp = 3, conf = 10, report = 'YXaSC')
colnames1 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df1_rules = pd.DataFrame(result1, columns = colnames1)
df1_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,whole milk,(),16.792521,100.000000,2335,16.792521,16.792521
1,whole milk,"(other vegetables,)",16.792521,12.980942,255,1.833873,14.127424
2,other vegetables,"(whole milk,)",12.980942,16.792521,255,1.833873,10.920771
3,other vegetables,(),12.980942,100.000000,1805,12.980942,12.980942
4,whole milk,"(rolls/buns,)",16.792521,11.708019,236,1.697231,14.496314
...,...,...,...,...,...,...,...
66,whole milk,"(coffee,)",16.792521,3.401654,65,0.467458,13.742072
67,other vegetables,"(coffee,)",12.980942,3.401654,48,0.345200,10.147992
68,whole milk,"(frozen vegetables,)",16.792521,3.013305,67,0.481841,15.990453
69,other vegetables,"(frozen vegetables,)",12.980942,3.013305,54,0.388350,12.887828


### Sorting and Filtering Rules

#### Sort Rules by Absolute Support
We will sort the generated rules by their absolute support in descending order. Sorting by support allows us to identify the most frequently occurring itemsets, which are likely to be the most relevant and actionable for the grocery store owner. We'll also filter out any rules with an empty antecedent to focus on meaningful associations where the presence of certain items influences the purchase of others.

In [ ]:
df1_rules = df1_rules.sort_values('absolute support', ascending = False)
df1_rules = df1_rules[df1_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df1_rules.shape)

(67, 7)


#### Display Top Rules by Support
We display the top 10 rules sorted by absolute support.  This will give us an idea of the most frequent and strong associations in our dataset, helping us identify key products that drive sales.

In [ ]:
df1_rules.head(10)

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,whole milk,"(other vegetables,)",16.792521,12.980942,255,1.833873,14.127424
1,other vegetables,"(whole milk,)",12.980942,16.792521,255,1.833873,10.920771
2,rolls/buns,"(whole milk,)",11.708019,16.792521,236,1.697231,10.107066
3,whole milk,"(rolls/buns,)",16.792521,11.708019,236,1.697231,14.496314
4,whole milk,"(soda,)",16.792521,10.370370,204,1.467098,14.147018
5,whole milk,"(yogurt,)",16.792521,9.176555,193,1.387990,15.125392
6,other vegetables,"(rolls/buns,)",12.980942,11.708019,177,1.272923,10.872236
7,other vegetables,"(soda,)",12.980942,10.370370,162,1.165049,11.234397
8,whole milk,"(sausage,)",16.792521,6.414959,154,1.107515,17.264574
9,whole milk,"(tropical fruit,)",16.792521,7.256383,147,1.057174,14.568880


#### Sort Rules by Confidence
Next, we sort the rules by confidence in descending order. Confidence measures the reliability of the association rules, indicating how often the consequent is purchased when the antecedent is present. Higher confidence values suggest stronger and more reliable associations. We also filter out any rules with an empty antecedent.

In [ ]:
df1_rules = df1_rules.sort_values('confidence (%)', ascending = False)
df1_rules = df1_rules[df1_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df1_rules.shape)

(67, 7)


#### Display Top Rules by Confidence
We display the top 10 rules sorted by confidence. This will help us identify the most reliable and confident associations in our dataset, providing actionable insights for improving marketing strategies and product placements.

In [ ]:
df1_rules.head(10)

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,whole milk,"(bottled beer,)",16.792521,4.847177,120,0.862999,17.804154
1,whole milk,"(beef,)",16.792521,3.646170,89,0.640058,17.554241
2,whole milk,"(sausage,)",16.792521,6.414959,154,1.107515,17.264574
3,whole milk,"(newspapers,)",16.792521,4.185545,94,0.676016,16.151203
4,whole milk,"(domestic eggs,)",16.792521,3.991370,89,0.640058,16.036036
5,whole milk,"(frozen vegetables,)",16.792521,3.013305,67,0.481841,15.990453
6,whole milk,"(frankfurter,)",16.792521,4.048903,90,0.647249,15.985790
7,whole milk,"(butter,)",16.792521,3.775620,83,0.596908,15.809524
8,whole milk,"(margarine,)",16.792521,3.459187,76,0.546566,15.800416
9,whole milk,"(citrus fruit,)",16.792521,5.695793,122,0.877382,15.404040


### Interpretation

#### Frequent Itemsets
- The most commonly purchased item by members on a monthly basis is whole milk (2335), followed by other vegetables (1805), rolls/buns (1628), soda (1442), and yogurt (1276). These items have high absolute support, indicating their popularity among customers.

#### Top Rules by Support
- "Whole milk" and "other vegetables" are frequently purchased together (support: 255, support (%): 1.83).
- "Whole milk" and "rolls/buns" are also commonly bought together (support: 236, support (%): 1.70).

#### Top Rules by Confidence
- Customers who buy "bottled beer" are likely to buy "whole milk" (confidence: 17.80%).
- Similarly, customers who buy "beef" or "sausage" are also likely to buy "whole milk" (confidence: 17.55% and 17.26%, respectively).

### Recommendations for the Owner

- Given that "whole milk" and "other vegetables" are frequently purchased together, consider placing these items closer to each other in the store. Additionally, consider creating bundled promotions that include both items to encourage further purchases.
- Since "whole milk" and "rolls/buns" are commonly bought together, consider cross-merchandising these items. For example, display rolls and buns near the dairy section to increase the likelihood of customers purchasing both items.
- Customers who buy "bottled beer" are likely to buy "whole milk." Create promotions or displays that pair these items together to capitalize on this association. Similarly, promote "beef" or "sausage" alongside "whole milk" to increase sales, as these items are also likely to be purchased together.
- Ensure there is ample stock of "whole milk," "other vegetables," "rolls/buns," "soda," and "yogurt," as these are consistently popular items. Keeping these items well-stocked can lead to increased sales and customer satisfaction.

### Other Groupings

Grouping the items by member and quarter also allows us to analyze purchasing patterns over longer periods, which may reveal different associations compared to monthly patterns. Quarterly grouping provides a broader view of customer behavior, capturing trends that may not be apparent in monthly data. This grouping also aligns with business operations and planning, as many businesses track performance and make strategic decisions on a quarterly basis.

#### Extract Quarter and Year
We extract the quarter from the 'Date' column and create a new column 'Quarter_Year'. This allows us to group the transactions by quarter and year, providing a different perspective on seasonal trends and purchasing patterns over longer periods.

In [ ]:
df1['Quarter_Year'] = df1['Date'].dt.to_period('Q')

#### Group Transactions by Member and Quarter
Similar to the monthly grouping, we group items bought by each member for each quarter and year. Grouping by quarter helps us analyze broader seasonal trends and identify long-term purchasing patterns. This grouped data will be reset to a DataFrame format for further analysis.

In [ ]:
grouped_member_quarter1 = df1.groupby(['Member_number', 'Quarter_Year'])['itemDescription'].apply(list).reset_index()
grouped_member_quarter1

,Member_number,Quarter_Year,itemDescription
0,1000,2014Q2,"[whole milk, pastry, salty snack]"
1,1000,2015Q1,"[sausage, whole milk, semi-finished bread, yog..."
2,1000,2015Q2,"[soda, pickled vegetables]"
3,1000,2015Q3,"[canned beer, misc. beverages]"
4,1000,2015Q4,"[sausage, hygiene articles]"
...,...,...,...
11921,4999,2015Q2,"[butter milk, whipped/sour cream]"
11922,4999,2015Q4,"[bottled water, herbs]"
11923,5000,2014Q1,"[fruit/vegetable juice, onions]"
11924,5000,2014Q4,"[bottled beer, other vegetables]"


This grouping resulted in 11926 rows, which is less than the number of rows when grouped by month.

#### Generate Frequent Itemsets for Quarterly Data
Using the apriori algorithm, we generate frequent itemsets from the quarterly grouped transactions with a minimum support threshold of 5. I used a higher support threshold of 5 instead of 3 because the number of rows has decreased, and some items were combined further with the quarterly grouping compared to the monthly grouping. This higher threshold ensures that only itemsets with significant support are considered, reflecting more consistent purchasing patterns over a longer period.

In [ ]:
itemsets1 = apriori(grouped_member_quarter1['itemDescription'], supp = 5)
df1_itemsets = pd.DataFrame(itemsets1, columns = ['itemset', 'absolute support'])
df1_itemsets = df1_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df1_itemsets.shape)

df1_itemsets

(16, 2)


,itemset,absolute support
0,"(whole milk,)",2273
1,"(other vegetables,)",1770
2,"(rolls/buns,)",1602
3,"(soda,)",1420
4,"(yogurt,)",1255
5,"(root vegetables,)",1025
6,"(tropical fruit,)",1000
7,"(bottled water,)",893
8,"(sausage,)",879
9,"(citrus fruit,)",784


#### Generate Association Rules for Quarterly Data
Next, we generate association rules from the frequent itemsets of quarterly data with a minimum support of 5 and minimum confidence of 10 to focus on meaningful associations. These rules will be stored in a DataFrame for further analysis, helping us understand the associations over a longer period.

In [ ]:
result1 = arules(grouped_member_quarter1['itemDescription'], supp = 5, conf = 10, report = 'YXaSC')
colnames1 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df1_rules = pd.DataFrame(result1, columns = colnames1)
df1_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,whole milk,(),19.059198,100.000000,2273,19.059198,19.059198
1,whole milk,"(other vegetables,)",19.059198,14.841523,324,2.716753,18.305085
2,other vegetables,"(whole milk,)",14.841523,19.059198,324,2.716753,14.254289
3,other vegetables,(),14.841523,100.000000,1770,14.841523,14.841523
4,whole milk,"(rolls/buns,)",19.059198,13.432836,313,2.624518,19.538077
...,...,...,...,...,...,...,...
73,yogurt,"(shopping bags,)",10.523227,5.928224,72,0.603723,10.183876
74,whole milk,"(whipped/sour cream,)",19.059198,5.425122,110,0.922355,17.001546
75,other vegetables,"(whipped/sour cream,)",14.841523,5.425122,95,0.796579,14.683153
76,rolls/buns,"(whipped/sour cream,)",13.432836,5.425122,69,0.578568,10.664606


#### Sort Quarterly Rules by Absolute Support
We sort the quarterly rules by absolute support in descending order and filter out rules with an empty antecedent. Sorting by support helps us focus on the most frequent itemsets, which are likely to be the most relevant for the store's inventory and marketing strategies.

In [ ]:
df1_rules = df1_rules.sort_values('absolute support', ascending = False)
df1_rules = df1_rules[df1_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df1_rules.shape)

(73, 7)


#### Display Top Quarterly Rules by Support
We display the top 10 rules sorted by absolute support. This will give us insights into the most frequent associations over a longer period, helping us identify key seasonal trends.

In [ ]:
df1_rules.head(10)

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,other vegetables,"(whole milk,)",14.841523,19.059198,324,2.716753,14.254289
1,whole milk,"(other vegetables,)",19.059198,14.841523,324,2.716753,18.305085
2,whole milk,"(rolls/buns,)",19.059198,13.432836,313,2.624518,19.538077
3,rolls/buns,"(whole milk,)",13.432836,19.059198,313,2.624518,13.770348
4,soda,"(whole milk,)",11.906758,19.059198,254,2.129800,11.174659
5,whole milk,"(soda,)",19.059198,11.906758,254,2.129800,17.887324
6,rolls/buns,"(other vegetables,)",13.432836,14.841523,232,1.945330,13.107345
7,other vegetables,"(rolls/buns,)",14.841523,13.432836,232,1.945330,14.481898
8,whole milk,"(yogurt,)",19.059198,10.523227,231,1.936944,18.406375
9,yogurt,"(whole milk,)",10.523227,19.059198,231,1.936944,10.162780


#### Sort Quarterly Rules by Confidence
We sort the quarterly rules by confidence in descending order and filter out rules with an empty antecedent. Sorting by confidence helps us focus on the most reliable associations, providing actionable insights for improving business strategies.

In [ ]:
df1_rules = df1_rules.sort_values('confidence (%)', ascending = False)
df1_rules = df1_rules[df1_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df1_rules.shape)

(73, 7)


#### Display Top Quarterly Rules by Confidence
We display the top 10 rules sorted by confidence. This will help us identify the most reliable and confident associations in the quarterly data, providing insights into long-term purchasing patterns.

In [ ]:
df1_rules.head(10)

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,whole milk,"(sausage,)",19.059198,7.370451,192,1.609928,21.843003
1,whole milk,"(bottled beer,)",19.059198,5.609592,145,1.215831,21.674141
2,whole milk,"(canned beer,)",19.059198,5.844374,141,1.182291,20.229555
3,whole milk,"(tropical fruit,)",19.059198,8.385041,196,1.643468,19.600000
4,whole milk,"(rolls/buns,)",19.059198,13.432836,313,2.624518,19.538077
5,whole milk,"(bottled water,)",19.059198,7.487842,173,1.450612,19.372900
6,whole milk,"(pip fruit,)",19.059198,6.062385,137,1.148751,18.948824
7,whole milk,"(shopping bags,)",19.059198,5.928224,133,1.115210,18.811881
8,whole milk,"(citrus fruit,)",19.059198,6.573872,146,1.224216,18.622449
9,whole milk,"(pastry,)",19.059198,6.406171,141,1.182291,18.455497


### Interpretation

#### Frequent Itemsets
- The most commonly purchased item by members on a quarterly basis is still whole milk (2273), followed by other vegetables (1770), rolls/buns (1602), soda (1420), and yogurt (1255). These items have high absolute support, indicating their popularity among customers.
- The consistent popularity of "whole milk," "other vegetables," "rolls/buns," "soda," and "yogurt" as the most commonly purchased items by members on both a monthly and quarterly basis indicates these items are essential and in high demand among customers.

#### Top Rules by Support
- As the size of the subset has changed with the quarterly grouping, the support got higher.
- "Whole milk" and "other vegetables" are frequently purchased together (support: 324, support (%): 2.72).
- "Whole milk" and "rolls/buns" are also commonly bought together (support: 313, support (%): 2.62).

#### Top Rules by Confidence
- As the size of the subset has changed with the quarterly grouping, the confidence got higher.
- Customers who buy "sausage" are likely to buy "whole milk" (confidence: 21.84%).
- Customers who buy "bottled beer" or "canned beer" are also likely to buy "whole milk" (confidence: 21.67% and 20.23%, respectively).

### Recommendations for the Owner

- Place "whole milk" close to "other vegetables" and "rolls/buns" to encourage additional purchases. Customers who buy one of these items are likely to buy the others as well.
- Offer discounts or create bundles featuring "whole milk" with "sausage," "bottled beer," or "canned beer." This can capitalize on the high likelihood of customers purchasing these items together.
- Ensure there is ample stock of "whole milk," "other vegetables," "rolls/buns," "soda," and "yogurt," as these are consistently popular items. Keeping these items well-stocked can lead to increased sales and customer satisfaction.

## Bakery Dataset

### Prepare the Dataset

We will be utilizing the <a href="https://www.kaggle.com/datasets/akashdeepkuila/bakery/data">Bakery Sales dataset</a> from Kaggle. This dataset contains 20507 rows.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df2 = pd.read_csv('/content/drive/My Drive/Bakery.csv')
df2

,TransactionNo,Items,DateTime,Daypart,DayType
0,1,Bread,2016-10-30 09:58:11,Morning,Weekend
1,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend
2,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend
3,3,Hot chocolate,2016-10-30 10:07:57,Morning,Weekend
4,3,Jam,2016-10-30 10:07:57,Morning,Weekend
...,...,...,...,...,...
20502,9682,Coffee,2017-09-04 14:32:58,Afternoon,Weekend
20503,9682,Tea,2017-09-04 14:32:58,Afternoon,Weekend
20504,9683,Coffee,2017-09-04 14:57:06,Afternoon,Weekend
20505,9683,Pastry,2017-09-04 14:57:06,Afternoon,Weekend


Using this dataset for association rule mining can help identify patterns and relationships between items frequently purchased together, enabling the bakery to optimize product placement and create effective marketing strategies. This analysis can uncover popular item pairings, peak sales times, and customer purchasing behaviors, providing valuable insights to enhance sales and customer satisfaction.

### Data Dictionary
Below is a brief description of each column in the dataset, including 'TransactionNo', 'Items', 'DateTime', 'Daypart', and 'DayType'.

| Variable Name | Description           | Data Type | Allowed Values / Format | Missing Values |
|------|-------------------------------------|--------|------------|----|
| TransactionNo | Unique identifier for each transaction | Integer | Positive integers | No |
| Items | Items purchased | String |  | No |
| DateTime | Date and time of the transaction | DateTime | YYYY-MM-DD HH:MM:SS | No |
| Daypart | Part of the day when the transaction occurrred | String | Morning, Afternoon, Evening, Night | No |
| DayType | Whether the transaction occurred on a weekday or weekend | String | Weekday, Weekend | No |

### Data Preprocessing

Let us first examine the data types of the variables.

In [ ]:
df2.dtypes

TransactionNo     int64
Items            object
DateTime         object
Daypart          object
DayType          object
dtype: object

#### Convert DateTime Column
We convert the 'DateTime' column from string format to DateTime objects using the pd.to_datetime function. This conversion is crucial for accurate date and time manipulation.

In [ ]:
df2['DateTime'] = pd.to_datetime(df2['DateTime'])
df2.dtypes

TransactionNo             int64
Items                    object
DateTime         datetime64[ns]
Daypart                  object
DayType                  object
dtype: object

#### Basic Statistics

#### Non-null Values
Check the count of non-null values in each column to ensure there are no missing values in the dataset.

In [ ]:
df2.count()

TransactionNo    20507
Items            20507
DateTime         20507
Daypart          20507
DayType          20507
dtype: int64

All columns contain 20507 entries, meaning that there are no missing values.

#### Null Values
Verify that there are no null values in any of the columns using the isnull().sum() method.

In [ ]:
df2.isnull().sum()

TransactionNo    0
Items            0
DateTime         0
Daypart          0
DayType          0
dtype: int64

#### Unique Values
Check the number of unique values in 'TransactionNo', 'Items', 'DateTime', 'Daypart', and 'DayType' columns to understand the dataset's structure. This helps us identify the diversity and range of data in each column.

In [ ]:
df2['TransactionNo'].nunique()

9465

In [ ]:
df2['TransactionNo'].unique()

array([   1,    2,    3, ..., 9682, 9683, 9684])

There are 9465 different transaction numbers.

In [ ]:
df2['Items'].nunique()

94

In [ ]:
df2['Items'].unique()

array(['Bread', 'Scandinavian', 'Hot chocolate', 'Jam', 'Cookies',
       'Muffin', 'Coffee', 'Pastry', 'Medialuna', 'Tea', 'Tartine',
       'Basket', 'Mineral water', 'Farm House', 'Fudge', 'Juice',
       "Ella's Kitchen Pouches", 'Victorian Sponge', 'Frittata',
       'Hearty & Seasonal', 'Soup', 'Pick and Mix Bowls', 'Smoothies',
       'Cake', 'Mighty Protein', 'Chicken sand', 'Coke',
       'My-5 Fruit Shoot', 'Focaccia', 'Sandwich', 'Alfajores', 'Eggs',
       'Brownie', 'Dulce de Leche', 'Honey', 'The BART', 'Granola',
       'Fairy Doors', 'Empanadas', 'Keeping It Local', 'Art Tray',
       'Bowl Nic Pitt', 'Bread Pudding', 'Adjustment', 'Truffles',
       'Chimichurri Oil', 'Bacon', 'Spread', 'Kids biscuit', 'Siblings',
       'Caramel bites', 'Jammie Dodgers', 'Tiffin', 'Olum & polenta',
       'Polenta', 'The Nomad', 'Hack the stack', 'Bakewell',
       'Lemon and coconut', 'Toast', 'Scone', 'Crepes', 'Vegan mincepie',
       'Bare Popcorn', 'Muesli', 'Crisps', 'Pintxos', 

There are 94 different items sold by the bakery.

In [ ]:
df2['DateTime'].nunique()

9465

In [ ]:
df2['DateTime'].unique()

<DatetimeArray>
['2016-10-30 09:58:11', '2016-10-30 10:05:34', '2016-10-30 10:07:57',
 '2016-10-30 10:08:41', '2016-10-30 10:13:03', '2016-10-30 10:16:55',
 '2016-10-30 10:19:12', '2016-10-30 10:20:51', '2016-10-30 10:21:59',
 '2016-10-30 10:25:58',
 ...
 '2017-09-04 13:04:08', '2017-09-04 13:06:34', '2017-09-04 13:20:23',
 '2017-09-04 13:49:21', '2017-09-04 14:08:37', '2017-09-04 14:24:03',
 '2017-09-04 14:30:09', '2017-09-04 14:32:58', '2017-09-04 14:57:06',
 '2017-09-04 15:04:24']
Length: 9465, dtype: datetime64[ns]

There are 9465 different date-times of transactions.

In [ ]:
df2['Daypart'].nunique()

4

In [ ]:
df2['Daypart'].unique()

array(['Morning', 'Afternoon', 'Evening', 'Night'], dtype=object)

There are 4 different parts of the day, which classifies whether the transaction occurred in the morning, afternoon, evening, or night.

In [ ]:
df2['DayType'].nunique()

2

In [ ]:
df2['DayType'].unique()

array(['Weekend', 'Weekday'], dtype=object)

There are 2 different types of day, which classifies whether the transaction occurred on a weekday or weekend.

#### Mode
Identify the mode (most frequent value) for 'TransactionNo', 'Items', 'DateTime', 'Daypart', and 'DayType' columns. This helps us understand the most common values and behaviors in our dataset.

In [ ]:
df2['TransactionNo'].mode()

0    6279
1    6412
2    6474
3    6716
Name: TransactionNo, dtype: int64

Transaction Numbers 6279, 6412, 6474, and 6716 had the most items bought.

In [ ]:
df2['Items'].mode()

0    Coffee
Name: Items, dtype: object

Coffee is the top selling item in the bakery.

In [ ]:
df2['DateTime'].mode()

0   2017-02-17 14:18:20
1   2017-09-02 13:44:56
2   2017-11-02 14:08:27
3   2017-12-02 14:35:34
Name: DateTime, dtype: datetime64[ns]

There are 4 date-times corresponding to 1 transaction each that had the most items bought.

In [ ]:
df2['Daypart'].mode()

0    Afternoon
Name: Daypart, dtype: object

Most items are bought in the afternoon.

In [ ]:
df2['DayType'].mode()

0    Weekday
Name: DayType, dtype: object

Most items are bought on a weekday.

### Grouping Items

#### Group Transactions by Transaction Number
We group items bought in each transaction using the groupby method and reset the grouped data to a DataFrame format. This step helps us aggregate transactions and analyze purchasing patterns at the transaction level.

In [ ]:
grouped_transaction2 = df2.groupby(['TransactionNo'])['Items'].apply(list).reset_index()
grouped_transaction2

,TransactionNo,Items
0,1,[Bread]
1,2,"[Scandinavian, Scandinavian]"
2,3,"[Hot chocolate, Jam, Cookies]"
3,4,[Muffin]
4,5,"[Coffee, Pastry, Bread]"
...,...,...
9460,9680,[Bread]
9461,9681,"[Truffles, Tea, Spanish Brunch, Christmas common]"
9462,9682,"[Muffin, Tacos/Fajita, Coffee, Tea]"
9463,9683,"[Coffee, Pastry]"


There is a total of 9465 transactions recorded.

### Calculate the Association Rules

#### Generate Frequent Itemsets
Using the apriori algorithm, we generate frequent itemsets from the grouped transactions with a minimum support threshold of 3. This threshold ensures that only itemsets appearing in at least 3% of all the transactions are considered, focusing on meaningful associations.

In [ ]:
itemsets2 = apriori(grouped_transaction2['Items'], supp = 3)
df2_itemsets = pd.DataFrame(itemsets2, columns = ['itemset', 'absolute support'])
df2_itemsets = df2_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df2_itemsets.shape)

df2_itemsets

(23, 2)


,itemset,absolute support
0,"(Coffee,)",4528
1,"(Bread,)",3097
2,"(Tea,)",1350
3,"(Cake,)",983
4,"(Bread, Coffee)",852
5,"(Pastry,)",815
6,"(Sandwich,)",680
7,"(Medialuna,)",585
8,"(Hot chocolate,)",552
9,"(Cake, Coffee)",518


#### Generate Association Rules
We generate association rules from the frequent itemsets of bakery sales data with a minimum support of 3 and minimum confidence of 50. The higher confidence threshold helps us focus on more reliable associations, providing actionable insights.

In [ ]:
result2 = arules(grouped_transaction2['Items'], supp = 3, conf = 50, report = 'YXaSC')
colnames2 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df2_rules = pd.DataFrame(result2, columns = colnames2)
df2_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Cake,)",47.839408,10.385631,518,5.472795,52.695829
1,Coffee,"(Pastry,)",47.839408,8.610671,450,4.754358,55.214724
2,Coffee,"(Sandwich,)",47.839408,7.184363,362,3.824617,53.235294
3,Coffee,"(Hot chocolate,)",47.839408,5.832013,280,2.958267,50.724638
4,Coffee,"(Medialuna,)",47.839408,6.180666,333,3.518225,56.923077
5,Coffee,"(Cookies,)",47.839408,5.441099,267,2.820919,51.844660
6,Coffee,"(Juice,)",47.839408,3.856313,195,2.060222,53.424658
7,Coffee,"(Alfajores,)",47.839408,3.634443,186,1.965135,54.069767
8,Coffee,"(Scone,)",47.839408,3.454834,171,1.806656,52.293578
9,Coffee,"(Toast,)",47.839408,3.359746,224,2.366614,70.440252


### Sorting and Filtering Rules

#### Sort Bakery Sales Rules by Absolute Support
We sort the bakery sales rules by absolute support in descending order and filter out rules with an empty antecedent. Sorting by support helps us identify the most frequent itemsets, which are likely to be the most relevant for the bakery.

In [ ]:
df2_rules = df2_rules.sort_values('absolute support', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)

(10, 7)


#### Display Top Rules by Support
We display the top 10 rules sorted by absolute support. This will give us insights into the most frequent associations in the bakery sales data, helping us identify key products that drive sales.

In [ ]:
df2_rules.head(10)

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Cake,)",47.839408,10.385631,518,5.472795,52.695829
1,Coffee,"(Pastry,)",47.839408,8.610671,450,4.754358,55.214724
2,Coffee,"(Sandwich,)",47.839408,7.184363,362,3.824617,53.235294
3,Coffee,"(Medialuna,)",47.839408,6.180666,333,3.518225,56.923077
4,Coffee,"(Hot chocolate,)",47.839408,5.832013,280,2.958267,50.724638
5,Coffee,"(Cookies,)",47.839408,5.441099,267,2.820919,51.844660
6,Coffee,"(Toast,)",47.839408,3.359746,224,2.366614,70.440252
7,Coffee,"(Juice,)",47.839408,3.856313,195,2.060222,53.424658
8,Coffee,"(Alfajores,)",47.839408,3.634443,186,1.965135,54.069767
9,Coffee,"(Scone,)",47.839408,3.454834,171,1.806656,52.293578


#### Sort Bakery Sales Rules by Confidence
We sort the bakery sales rules by confidence in descending order and filter out rules with an empty antecedent. Sorting by confidence helps us focus on the most reliable associations, providing actionable insights for improving business strategies.

In [ ]:
df2_rules = df2_rules.sort_values('confidence (%)', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)

(10, 7)


#### Display Top Rules by Confidence
We display the top 10 rules sorted by confidence. This will help us identify the most reliable and confident associations in the bakery sales data, providing insights into customer purchasing behavior.

In [ ]:
df2_rules.head(10)

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Toast,)",47.839408,3.359746,224,2.366614,70.440252
1,Coffee,"(Medialuna,)",47.839408,6.180666,333,3.518225,56.923077
2,Coffee,"(Pastry,)",47.839408,8.610671,450,4.754358,55.214724
3,Coffee,"(Alfajores,)",47.839408,3.634443,186,1.965135,54.069767
4,Coffee,"(Juice,)",47.839408,3.856313,195,2.060222,53.424658
5,Coffee,"(Sandwich,)",47.839408,7.184363,362,3.824617,53.235294
6,Coffee,"(Cake,)",47.839408,10.385631,518,5.472795,52.695829
7,Coffee,"(Scone,)",47.839408,3.454834,171,1.806656,52.293578
8,Coffee,"(Cookies,)",47.839408,5.441099,267,2.820919,51.844660
9,Coffee,"(Hot chocolate,)",47.839408,5.832013,280,2.958267,50.724638


### Interpretation

#### Frequent Itemsets
- The most frequent items sold individually are Coffee (4528), Bread (3097), and Tea (1350).
- The top item pairs include Bread & Coffee (852), Cake & Coffee (518), and Tea & Coffee (472).

#### Top Rules by Support
- Coffee is a frequent consequent item.
- 518 Cake purchases also include Coffee.
- 450 Pastry purchases also include Coffee.
- 362 Sandwich purchases also include Coffee.

#### Top Rules by Confidence
- Coffee has high confidence with certain items.
- 70.4% of Toast purchases also include Coffee.
- 56.9% of Medialuna purchases also include Coffee.
- 55.2% of Pastry purchases also include Coffee.

### Recommendations for the Owner

- Since Coffee is frequently bought with Cake, Pastry, Sandwich, Medialuna, and Toast, create combo deals or special promotions that pair Coffee with these items. For example, offer a discount when customers purchase Coffee with any of these items.
- Arrange displays to encourage customers to buy these pairs. For instance, place Coffee near the display of Cake, Pastry, Sandwiches, and Toast.
- Use signage to suggest pairing Coffee with Cake, Pastry, Sandwiches, and Toast, emphasizing the frequent and confident pairings.
- Ensure adequate inventory of Coffee, Bread, and Tea, as they have high individual sales. Also, maintain a good stock of Cake, Pastry, Toast, Sandwiches, and Medialuna, given their frequent pairing with Coffee.
- Train staff to suggest popular pairings. For instance, when a customer orders a Medialuna, the staff could suggest adding a Coffee.

### Other Groupings

To uncover time-specific purchasing patterns, let us also try filtering the sales by part of the day (morning, afternoon, evening, night) and recalculating the rules. This approach allows for a more granular understanding of customer behavior, revealing how item pairings and purchasing habits vary throughout the day, and enabling the bakery to tailor promotions and inventory management to meet the specific demands of different dayparts.

#### Extract Unique Dayparts
Display the unique values in the 'Daypart' column to identify the different parts of the day.

In [ ]:
df2['Daypart'].unique()

array(['Morning', 'Afternoon', 'Evening', 'Night'], dtype=object)

#### Group Items for Morning Transactions
Group items bought in morning transactions using the groupby method and reset the grouped data to a DataFrame format.

In [ ]:
morning = df2[df2['Daypart'] == 'Morning']
grouped_morning = morning.groupby(['TransactionNo'])['Items'].apply(list).reset_index(name = 'Item_List')

#### Group Items for Afternoon Transactions
Group items bought in afternoon transactions using the groupby method and reset the grouped data to a DataFrame format.

In [ ]:
afternoon = df2[df2['Daypart'] == 'Afternoon']
grouped_afternoon = afternoon.groupby(['TransactionNo'])['Items'].apply(list).reset_index(name = 'Item_List')

#### Group Items for Evening Transactions
Group items bought in evening transactions using the groupby method and reset the grouped data to a DataFrame format.

In [ ]:
evening = df2[df2['Daypart'] == 'Evening']
grouped_evening = evening.groupby(['TransactionNo'])['Items'].apply(list).reset_index(name = 'Item_List')

#### Group Items for Night Transactions
Group items bought in night transactions using the groupby method and reset the grouped data to a DataFrame format.

In [ ]:
night = df2[df2['Daypart'] == 'Night']
grouped_night = night.groupby(['TransactionNo'])['Items'].apply(list).reset_index(name = 'Item_List')

#### Calculate the Association Rules for Morning Transactions
Generate frequent itemsets and association rules for morning transactions. Since the subset has become smaller after filtering the morning transactions only, let us use a lower support threshold of 1 and a lower confidence threshold of 20. Sort and display the top rules by support and confidence.

In [ ]:
itemsets2 = apriori(grouped_morning['Item_List'], supp = 1)
df2_itemsets = pd.DataFrame(itemsets2, columns = ['itemset', 'absolute support'])
df2_itemsets = df2_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df2_itemsets.shape)

df2_itemsets

(49, 2)


,itemset,absolute support
0,"(Coffee,)",2113
1,"(Bread,)",1490
2,"(Pastry,)",572
3,"(Tea,)",441
4,"(Bread, Coffee)",386
5,"(Medialuna,)",380
6,"(Pastry, Coffee)",317
7,"(Cake,)",259
8,"(Medialuna, Coffee)",224
9,"(Hot chocolate,)",217


In [ ]:
result2 = arules(grouped_morning['Item_List'], supp = 1, conf = 20, report = 'YXaSC')
colnames2 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df2_rules = pd.DataFrame(result2, columns = colnames2)
df2_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,(),51.498903,100.000000,2113,51.498903,51.498903
1,Coffee,"(Bread,)",51.498903,36.314892,386,9.407750,25.906040
2,Bread,(),36.314892,100.000000,1490,36.314892,36.314892
3,Coffee,"(Pastry,)",51.498903,13.941019,317,7.726054,55.419580
4,Coffee,"(Pastry, Bread)",51.498903,4.703875,66,1.608579,34.196891
...,...,...,...,...,...,...,...
60,Coffee,"(Keeping It Local,)",51.498903,1.194248,38,0.926152,77.551020
61,Bread,"(Keeping It Local,)",36.314892,1.194248,16,0.389959,32.653061
62,Tea,"(Keeping It Local,)",10.748233,1.194248,11,0.268097,22.448980
63,Coffee,"(Fudge,)",51.498903,1.121131,12,0.292469,26.086957


In [ ]:
df2_rules = df2_rules.sort_values('absolute support', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(63, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Bread,)",51.498903,36.314892,386,9.407750,25.906040
1,Coffee,"(Pastry,)",51.498903,13.941019,317,7.726054,55.419580
2,Coffee,"(Medialuna,)",51.498903,9.261516,224,5.459420,58.947368
3,Bread,"(Pastry,)",36.314892,13.941019,193,4.703875,33.741259
4,Coffee,"(Tea,)",51.498903,10.748233,168,4.094565,38.095238
5,Coffee,"(Toast,)",51.498903,4.971972,147,3.582744,72.058824
6,Coffee,"(Cake,)",51.498903,6.312454,136,3.314648,52.509653
7,Bread,"(Medialuna,)",36.314892,9.261516,120,2.924689,31.578947
8,Coffee,"(Cookies,)",51.498903,4.752620,117,2.851572,60.000000
9,Coffee,"(Hot chocolate,)",51.498903,5.288813,117,2.851572,53.917051


In [ ]:
df2_rules = df2_rules.sort_values('confidence (%)', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(63, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Keeping It Local,)",51.498903,1.194248,38,0.926152,77.551020
1,Coffee,"(Toast,)",51.498903,4.971972,147,3.582744,72.058824
2,Coffee,"(Spanish Brunch,)",51.498903,1.169876,34,0.828662,70.833333
3,Coffee,"(Juice,)",51.498903,3.192786,80,1.949793,61.068702
4,Coffee,"(Cookies,)",51.498903,4.752620,117,2.851572,60.000000
5,Coffee,"(Alfajores,)",51.498903,2.461613,60,1.462345,59.405941
6,Coffee,"(Tiffin,)",51.498903,1.194248,29,0.706800,59.183673
7,Coffee,"(Medialuna,)",51.498903,9.261516,224,5.459420,58.947368
8,Coffee,"(Pastry,)",51.498903,13.941019,317,7.726054,55.419580
9,Coffee,"(Scone,)",51.498903,2.656593,60,1.462345,55.045872


#### Calculate the Association Rules for Afternoon Transactions
Generate frequent itemsets and association rules for afternoon transactions. Since the subset has become smaller after filtering the afternoon transactions only, let us use a lower support threshold of 1 and a lower confidence threshold of 20. Sort and display the top rules by support and confidence.

In [ ]:
itemsets2 = apriori(grouped_afternoon['Item_List'], supp = 1)
df2_itemsets = pd.DataFrame(itemsets2, columns = ['itemset', 'absolute support'])
df2_itemsets = df2_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df2_itemsets.shape)

df2_itemsets

(70, 2)


,itemset,absolute support
0,"(Coffee,)",2340
1,"(Bread,)",1556
2,"(Tea,)",864
3,"(Cake,)",696
4,"(Sandwich,)",590
...,...,...
65,"(Jam,)",58
66,"(Salad, Coffee)",57
67,"(Smoothies,)",55
68,"(Scone, Bread)",52


In [ ]:
result2 = arules(grouped_afternoon['Item_List'], supp = 1, conf = 20, report = 'YXaSC')
colnames2 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df2_rules = pd.DataFrame(result2, columns = colnames2)
df2_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,(),45.981529,100.000000,2340,45.981529,45.981529
1,Coffee,"(Bread,)",45.981529,30.575752,458,8.999803,29.434447
2,Bread,(),30.575752,100.000000,1556,30.575752,30.575752
3,Coffee,"(Tea,)",45.981529,16.977795,291,5.718216,33.680556
4,Coffee,"(Tea, Bread)",45.981529,3.379839,46,0.903910,26.744186
...,...,...,...,...,...,...,...
104,Bread,"(Smoothies,)",30.575752,1.080762,17,0.334054,30.909091
105,Cake,"(Smoothies,)",13.676557,1.080762,13,0.255453,23.636364
106,Sandwich,"(Smoothies,)",11.593633,1.080762,17,0.334054,30.909091
107,Bread,"(Jam,)",30.575752,1.139713,16,0.314404,27.586207


In [ ]:
df2_rules = df2_rules.sort_values('absolute support', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(107, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Bread,)",45.981529,30.575752,458,8.999803,29.434447
1,Coffee,"(Cake,)",45.981529,13.676557,366,7.191983,52.586207
2,Coffee,"(Sandwich,)",45.981529,11.593633,317,6.229122,53.728814
3,Coffee,"(Tea,)",45.981529,16.977795,291,5.718216,33.680556
4,Tea,"(Cake,)",16.977795,13.676557,166,3.261938,23.850575
5,Bread,"(Cake,)",30.575752,13.676557,155,3.045785,22.270115
6,Coffee,"(Hot chocolate,)",45.981529,6.150521,154,3.026135,49.201278
7,Coffee,"(Cookies,)",45.981529,5.895068,141,2.770682,47.000000
8,Coffee,"(Soup,)",45.981529,6.071920,141,2.770682,45.631068
9,Bread,"(Sandwich,)",30.575752,11.593633,135,2.652781,22.881356


In [ ]:
df2_rules = df2_rules.sort_values('confidence (%)', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(107, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Toast,)",45.981529,2.240126,77,1.513067,67.543860
1,Coffee,"(Sandwich, Cake)",45.981529,1.198664,41,0.805659,67.213115
2,Coffee,"(Soup, Sandwich)",45.981529,1.002162,33,0.648457,64.705882
3,Coffee,"(Salad,)",45.981529,1.748870,57,1.120063,64.044944
4,Coffee,"(Hot chocolate, Cake)",45.981529,1.493417,47,0.923561,61.842105
5,Coffee,"(Pastry,)",45.981529,4.578503,130,2.554529,55.793991
6,Coffee,"(Spanish Brunch,)",45.981529,2.436628,69,1.355866,55.645161
7,Coffee,"(Hearty & Seasonal,)",45.981529,1.788171,50,0.982511,54.945055
8,Coffee,"(Sandwich,)",45.981529,11.593633,317,6.229122,53.728814
9,Coffee,"(Medialuna,)",45.981529,3.733543,102,2.004323,53.684211


#### Calculate the Association Rules for Evening Transactions
Generate frequent itemsets and association rules for evening transactions. Since the subset has become smaller after filtering the evening transactions only, let us use a lower support threshold of 1 and a lower confidence threshold of 20. Sort and display the top rules by support and confidence.

In [ ]:
itemsets2 = apriori(grouped_evening['Item_List'], supp = 1)
df2_itemsets = pd.DataFrame(itemsets2, columns = ['itemset', 'absolute support'])
df2_itemsets = df2_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df2_itemsets.shape)

df2_itemsets

(82, 2)


,itemset,absolute support
0,"(Coffee,)",75
1,"(Bread,)",51
2,"(Tea,)",45
3,"(Cake,)",28
4,"(Hot chocolate,)",21
...,...,...
77,"(Mineral water, Alfajores)",3
78,"(Hot chocolate, Cookies)",3
79,"(Pastry, Bread)",3
80,"(Hot chocolate, Cake, Coffee)",3


In [ ]:
result2 = arules(grouped_evening['Item_List'], supp = 1, conf = 20, report = 'YXaSC')
colnames2 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df2_rules = pd.DataFrame(result2, columns = colnames2)
df2_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,(),28.735632,100.000000,75,28.735632,28.735632
1,Coffee,"(Tea,)",28.735632,17.241379,13,4.980843,28.888889
2,Coffee,"(Bread, Tea)",28.735632,3.831418,2,0.766284,20.000000
3,Tea,"(Bread, Coffee)",17.241379,3.065134,2,0.766284,25.000000
4,Bread,"(Tea,)",19.540230,17.241379,10,3.831418,22.222222
...,...,...,...,...,...,...,...
400,Farm House,"(Focaccia,)",3.448276,1.149425,1,0.383142,33.333333
401,Hack the stack,"(My-5 Fruit Shoot,)",0.383142,1.149425,1,0.383142,33.333333
402,Kids biscuit,"(My-5 Fruit Shoot,)",0.383142,1.149425,1,0.383142,33.333333
403,Extra Salami or Feta,"(Chicken Stew,)",0.383142,1.532567,1,0.383142,25.000000


In [ ]:
df2_rules = df2_rules.sort_values('absolute support', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(404, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Cake,)",28.735632,10.727969,16,6.130268,57.142857
1,Cake,"(Coffee,)",10.727969,28.735632,16,6.130268,21.333333
2,Coffee,"(Tea,)",28.735632,17.241379,13,4.980843,28.888889
3,Bread,"(Tea,)",19.540230,17.241379,10,3.831418,22.222222
4,Cake,"(Tea,)",10.727969,17.241379,9,3.448276,20.000000
5,Coffee,"(Hot chocolate,)",28.735632,8.045977,9,3.448276,42.857143
6,Coffee,"(Cookies,)",28.735632,7.662835,9,3.448276,45.000000
7,Coffee,"(Alfajores,)",28.735632,6.130268,9,3.448276,56.250000
8,Tea,"(Cake,)",17.241379,10.727969,9,3.448276,32.142857
9,Tea,"(Cookies,)",17.241379,7.662835,8,3.065134,40.000000


In [ ]:
df2_rules = df2_rules.sort_values('confidence (%)', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(404, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Coffee,"(Salad,)",28.735632,1.149425,3,1.149425,100.000000
1,Coffee,"(Scone,)",28.735632,1.532567,3,1.149425,75.000000
2,Alfajores,"(Mineral water,)",6.130268,1.532567,3,1.149425,75.000000
3,Tea,"(Scone,)",17.241379,1.532567,3,1.149425,75.000000
4,Coffee,"(Medialuna, Tea)",28.735632,1.532567,3,1.149425,75.000000
5,Cake,"(Alfajores, Bread)",10.727969,1.532567,3,1.149425,75.000000
6,Bread,"(Alfajores, Cake)",19.540230,1.532567,3,1.149425,75.000000
7,Coffee,"(Tiffin,)",28.735632,1.532567,3,1.149425,75.000000
8,Cake,"(Scone,)",10.727969,1.532567,3,1.149425,75.000000
9,Tea,"(Pastry, Coffee)",17.241379,1.149425,2,0.766284,66.666667


#### Calculate the Association Rules for Night Transactions
Generate frequent itemsets and association rules for night transactions. Since the subset has become smaller after filtering the night transactions only, let us use a lower support threshold of 1 and a lower confidence threshold of 20. Sort and display the top rules by support and confidence.

In [ ]:
itemsets2 = apriori(grouped_night['Item_List'], supp = 1)
df2_itemsets = pd.DataFrame(itemsets2, columns = ['itemset', 'absolute support'])
df2_itemsets = df2_itemsets.sort_values(by = 'absolute support', ascending = False).reset_index(drop = True)
print(df2_itemsets.shape)

df2_itemsets

(7, 2)


,itemset,absolute support
0,"(Vegan Feast,)",7
1,"(Valentine's card,)",2
2,"(Hot chocolate,)",1
3,"(Scandinavian,)",1
4,"(Juice, Mineral water)",1
5,"(Juice,)",1
6,"(Mineral water,)",1


In [ ]:
result2 = arules(grouped_night['Item_List'], supp = 1, conf = 20, report = 'YXaSC')
colnames2 = ['consequent', 'antecedent', 'consequent support (%)', 'antecedent support (%)', 'absolute support', 'relative support (%)', 'confidence (%)']
df2_rules = pd.DataFrame(result2, columns = colnames2)
df2_rules

,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Vegan Feast,(),58.333333,100.000000,7,58.333333,58.333333
1,Juice,"(Mineral water,)",8.333333,8.333333,1,8.333333,100.000000
2,Mineral water,"(Juice,)",8.333333,8.333333,1,8.333333,100.000000


In [ ]:
df2_rules = df2_rules.sort_values('absolute support', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(2, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Juice,"(Mineral water,)",8.333333,8.333333,1,8.333333,100.0
1,Mineral water,"(Juice,)",8.333333,8.333333,1,8.333333,100.0


In [ ]:
df2_rules = df2_rules.sort_values('confidence (%)', ascending = False)
df2_rules = df2_rules[df2_rules['antecedent'].apply(lambda x: len(x) > 0)].reset_index(drop = True)
print(df2_rules.shape)
df2_rules.head(10)

(2, 7)


,consequent,antecedent,consequent support (%),antecedent support (%),absolute support,relative support (%),confidence (%)
0,Juice,"(Mineral water,)",8.333333,8.333333,1,8.333333,100.0
1,Mineral water,"(Juice,)",8.333333,8.333333,1,8.333333,100.0


### Recommendations for the Owner

- Ensure adequate stock of Coffee and its frequent pairings like Bread, Pastry, Cake, and Sandwich to meet demand.
- Equip staff with knowledge of the most frequently paired items to enhance customer service and boost sales.
- Implement time-specific promotions tailored to morning, afternoon, evening, and night based on the identified frequent pairings, to drive sales during peak times.

#### Morning Strategies
- Create morning combo deals including Coffee and Bread, Pastry, or Medialuna.
- Promote "Keeping It Local" dishes, Toast, and Spanish Brunch with Coffee as part of a breakfast or brunch menu.

#### Afternoon Strategies
- Offer afternoon specials that bundle Coffee with Bread, Cake, or Sandwich.

#### Evening Strategies
- Offer evening discounts on Cake and Coffee or Tea and Coffee pairings.
- Highlight the pairing of Salad and Coffee or Scone and Coffee as evening snack options.

#### Night Strategies
- There seems to be few customers at night.
- Lower the prices of Juice and Mineral Water during nighttime hours to encourage purchases and increase sales.

## Conclusion

This analysis explored the use of association rule learning on the Groceries and Bakery Sales datasets to identify frequent itemsets and generate meaningful association rules. By applying the apriori and arules algorithms, important purchasing patterns and actionable insights were uncovered, which can inform business strategies.

In the Groceries dataset analysis, whole milk was found to be frequently bought with other vegetables and rolls/buns. Strong associations between items like bottled beer and whole milk were also discovered. Based on these findings, it is recommended to place frequently bought-together items closer to each other, create bundled promotions, and ensure adequate stock of popular items to boost sales and customer satisfaction.

For the Bakery Sales dataset, coffee was often purchased with items like cake, pastry, sandwich, medialuna, and toast. Analyzing transactions by daypart (morning, afternoon, evening, night) revealed time-specific purchasing patterns. These insights suggest opportunities for tailored promotions, optimized product placement, and better inventory management, enhancing the bakery’s ability to meet customer needs at different times of the day.

Overall, this analysis demonstrates the effectiveness of association rule learning in extracting valuable insights from transactional data. The findings provide a basis for making data-driven decisions that can improve customer satisfaction, increase sales, and streamline operations.

## Writeup/Analysis

Refer to the **Requirements section** for the guide questions.

## References

Cite any lectures, articles, notebooks etc. as resources.

- andrewm4894. (2020, September 29). Market basket analysis in Python. Andrewm4894. https://andrewm4894.com/2020/09/29/market-basket-analysis-in-python/
- Valdez, E. F. (2024, June 14). Association rule learning [Lecture]. Data Mining and Statistics. Retrieved from Course Materials.

## End
<sup>prepared by **Eduardo Valdez**</sup> <br>
<sup>for comments, corrections, suggestions, please email:</sup><sup>  <href>eduardo.valdez@dlsu.edu.ph</href></sup><br>